In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rapidfuzz
import Levenshtein

In [259]:
df1=pd.read_csv("/Users/corentinpla/Documents/Statap/transactions_A.csv")
df2=pd.read_csv("/Users/corentinpla/Documents/Statap/transactions_B.csv")

In [260]:
#pour identifier quelle row est avec quelle row
df1["matchA"]=df1.index
df2["matchB"]=df2.index

In [261]:
df_merge_perfect=pd.merge(df1,df2,on='tx_id')

In [230]:
df1=df1.drop(['tx_id','is_alert_solo','is_alert_pooled',"matchA"],axis=1)
df2=df2.drop(['tx_id','is_alert_solo','is_alert_pooled',"matchB"],axis=1)

In [192]:
#on ne garde que AB et BA
df1_AB=df1[df1["sender_bank_id"]=="A"]
df1_AB=df1_AB[df1_AB["receiver_bank_id"]=="B"]

df1_BA=df1[df1["sender_bank_id"]=="B"]
df1_BA=df1_BA[df1_BA["receiver_bank_id"]=="A"]

df1=pd.concat([df1_AB,df1_BA])

df2_AB=df2[df2["sender_bank_id"]=="A"]
df2_AB=df2_AB[df2_AB["receiver_bank_id"]=="B"]

df2_BA=df2[df2["sender_bank_id"]=="B"]
df2_BA=df2_BA[df2_BA["receiver_bank_id"]=="A"]

df2=pd.concat([df2_AB,df2_BA])

In [193]:
df1["sender_zip_code"]=df1["sender_zip_code"].apply(lambda x:str(x))
df1["sender_account_id"]=df1["sender_account_id"].apply(lambda x:str(x))
df1["receiver_account_id"]=df1["receiver_account_id"].apply(lambda x:str(x))
df1["receiver_zip_code"]=df1["receiver_zip_code"].apply(lambda x:str(x))
df1["tx_amount"]=df1["tx_amount"].apply(lambda x:str(x))

df2["sender_zip_code"]=df2["sender_zip_code"].apply(lambda x:str(x))
df2["sender_account_id"]=df2["sender_account_id"].apply(lambda x:str(x))
df2["receiver_account_id"]=df2["receiver_account_id"].apply(lambda x:str(x))
df2["receiver_zip_code"]=df2["receiver_zip_code"].apply(lambda x:str(x))
df2["tx_amount"]=df2["tx_amount"].apply(lambda x:str(x))

In [194]:
df1_bis=df1.copy()
df2_bis=df2.copy()

In [195]:
#prend pour argument les deux dataset et retourne une liste de sous dataset de pivot fixé

def get_pivot(piv,df_A,df_B):
    L_df=[]
    L_A=df_A[piv].drop_duplicates()
    L_B=df_B[piv].drop_duplicates()
    L_piv=list(set(L_A) & set(L_B)) #intersection des pivots des deux datasets si bruit = problème 


    for x in  (L_piv):
        df_A_piv=df_A[df_A[piv]==x]
        df_B_piv=df_B[df_B[piv]==x]
        L_df.append((df_A_piv,df_B_piv))
    
    return (L_df)

In [196]:
#dfA et dfB sont déjà des sub dataframes 
#pour un dataframe donné, retourne le dataframe sans la colonne pivot et avec la colonne "s" qui est 
#la concaténation des colonnes restantes. 

def get_string(dfA,dfB,pivot):
    dfA=dfA.drop(pivot,axis=1)
    dfB=dfB.drop(pivot,axis=1)
    n=len(dfA.columns)
    dfA["s"]=dfA[dfA.columns[0]]
    dfB["s"]=dfB[dfB.columns[0]]

    for k in range  (1,n) :
        dfA["s"]=dfA["s"]+dfA[dfA.columns[k]]
        dfB["s"]=dfB["s"]+dfB[dfB.columns[k]]
    
    return(dfA,dfB)

In [197]:
def get_distance_levenstein(dfA,dfB,rowA,rowB):
    sA=dfA.at[rowA,"s"]
    sB=dfB.at[rowB,"s"]
    return(Levenshtein.distance(sA,sB))

In [198]:
#dfA et dfB deux sub dataframe d'une liste (comme retourné par pivot)

def get_min_levenstein(dfA,dfB,rowA):
    min =get_distance_levenstein(dfA,dfB,rowA,dfB.index[0])
    dist=0
    ind=dfB.index[0]
    for y in dfB.index :
        dist =get_distance_levenstein(dfA,dfB,rowA,y)
        if dist <min :
            min=dist 
            ind=y
        
    return(ind,dist)

In [205]:
#dfA et dfB deux sub dataframe d'une liste (comme retourné par pivot)
#retourne les couples d'incice de dfA et dfB qui match le mieux

def get_merge(dfA,dfB):
    dfA["match"]=dfA.index
    dfB["match"]=dfB.index
    match=[]
    lev=0
    
    for x in dfA.index:
        lev =get_min_levenstein(dfA,dfB,x)[0]
        dfA.at[x,"match"]=lev
        match.append(x)
    
    
    
    df_merge=pd.merge(dfA,dfB,on='match')
    df_merge["matchA"]=match 
    df_merge=df_merge.rename(columns={"match": "matchB"})
    return(df_merge)
        


In [211]:
#fonction finale
#prend pour argument les deux dataframes, un pivot, et les merge

def pipe_merge(df_A,df_B,piv):
    
    L_pivot_1=get_pivot(piv,df_A,df_B)
    L_pivot_2=[]
    
    for x in L_pivot_1:
        x=get_string(x[0],x[1],piv)
        L_pivot_2.append(x)
    L_pivot_1=[]
    
    for x in L_pivot_2 : 
        L_pivot_1.append(get_merge(x[0],x[1]))
    
    return(pd.concatenate(L_pivot_1))

In [263]:
#prend pour argument le dataframe merge grace à Levenshtein et le dataframe merge grace à tx_id et retourne 
#le score du merge et les indices des lignes problématiques

def oracle (df_merge,df_perfect): 
    count_false=0
    row_false=[]
    df_merge=df_merge.set_index("matchA")
    df_perfect=df_perfect.set_index("matchA")
    
    for x in df_merge.index :
        if df_perfect.at[x,'matchB']!=df_merge.at[x,'matchB']:
            count_false+=1
            row_false.append([x,df_perfect.at[x,'matchB'],df_merge.at[x,'matchB']])
    return (count_false,row_false)
    
    

In [266]:
##tests
#pivot : sender_first_name
L_pivot=get_pivot('sender_first_name',df1_bis,df2_bis)

In [267]:
L_pivot_bis=[]
for x in L_pivot:
    x=get_string(x[0],x[1],'sender_first_name')
    L_pivot_bis.append(x)

In [268]:
len(L_pivot_bis)

1130

In [269]:
L_pivot=[]
counter=0
for x in L_pivot_bis :
    counter+=1
    print(counter)
    #print(x[0].shape)
    L_pivot.append(get_merge(x[0],x[1]))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [270]:
df_merge=pd.concat(L_pivot)

In [271]:
oracle(df_merge,df_merge_perfect)

(19287,
 [[287587, 157817, 146003],
  [287708, 158046, 146003],
  [290216, 159148, 158046],
  [290647, 160079, 159148],
  [292974, 159525, 160079],
  [294697, 161524, 159525],
  [295402, 162933, 161524],
  [298065, 163598, 162933],
  [298123, 161549, 163598],
  [300832, 165873, 161549],
  [302646, 165779, 165873],
  [303574, 167024, 165779],
  [304526, 166610, 167024],
  [304862, 167159, 166610],
  [308078, 170498, 167159],
  [308400, 169249, 170498],
  [308949, 169151, 169249],
  [310634, 168970, 169151],
  [311114, 170557, 168970],
  [311888, 172289, 170557],
  [311995, 172859, 172289],
  [312155, 172023, 172859],
  [312166, 172630, 172023],
  [312364, 172871, 172630],
  [312721, 172006, 172871],
  [313344, 171927, 172006],
  [313794, 171260, 171927],
  [313970, 172631, 171260],
  [314306, 171765, 172631],
  [314365, 172191, 171765],
  [314451, 172794, 172191],
  [314596, 171239, 172794],
  [314660, 172398, 171239],
  [314694, 171767, 172398],
  [315217, 172564, 171767],
  [315240, 1